# Human Presence Detection Training using Tensorflow


#### Click on "Kernel -> Restart & Run All" to start
#### Click on "Kernel -> Shutdown" before closing the window

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import yaml
import os
from collections import OrderedDict 
config_data = OrderedDict()
config_file = os.path.join(os.environ['CONFIG_FILE'], "config.txt")
if os.path.isfile(config_file):
    with open(config_file) as fp:
        config_data = yaml.load(fp.read(), Loader=yaml.FullLoader)
else:
    config_data["data_path"] = None
    config_data["train_path"] = None
    config_data["summary_step"] = "100"
    config_data["checkpoint_step"] = "500"
    config_data["max_steps"] = "250000"

### Path to user's dataset
#### The volume which is shared by the host to docker environment should contain the dataset and train logs directory.
#### Note that the dataset should be in KITTI format. Images and labels should be arranged as follows:
 - __Training Images:__ 'data_path'/training/images
 - __Labels:__ 'data_path'/training/labels
 - __train.txt:__'data_path'/ImageSets/train.txt
 
#### Click the following "Select dataset path" button to select dataset path from your shared directory in docker environment. The path of shared folder in docker environment will same be as that in host machine.

In [3]:
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
data_dir = None

class SelectPathButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""
    def __init__(self):
        super(SelectPathButton, self).__init__(layout=widgets.Layout(width='20%', height='30px'))
        
        self.description = "Select dataset path"
        self.on_click(self.select_path)

    @staticmethod
    def select_path(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askdirectory(initialdir = "/")
        if b.files != ():
            global data_dir
            data_dir = b.files
            path.value = data_dir
            config_data["data_path"] = data_dir
            with open(config_file, 'w') as outf:
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()
        

button = SelectPathButton()
if config_data["data_path"] is not None:
    path = widgets.Text(layout=widgets.Layout(width='30%'), value=config_data["data_path"])
    data_dir = config_data["data_path"]
else:
    path = widgets.Text(layout=widgets.Layout(width='30%'))

widgets.HBox([path, button])

### Training Configuration and Start Training
#### User can customize the training parameters by changing the following values. 
1. __Train directory:__ The volume which is shared by host to docker environment should contain a train logs directory. Using the "Select train directory" button select that path from docker. The path of shared folder in docker environment will same be as that in host machine.
2. __Summary steps:__ Number of steps to save summary
3. __Checkpoint step:__ Number of steps to save checkpoint
4. __Maximum steps:__ Maximum number of batches to run

In [4]:
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
train_dir = None
class SelectPathButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""
    def __init__(self):
        super(SelectPathButton, self).__init__(layout=widgets.Layout(width='20%', height='30px'))
        # Create the button.
        self.description = "Select train directory"
        self.on_click(self.select_path)

    @staticmethod
    def select_path(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askdirectory(initialdir = "/")
        if b.files != ():
            global train_dir
            train_dir = b.files
            train_path.value = train_dir
            config_data["train_path"] = train_dir
            with open(config_file, 'w') as outf:
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()

train_button = SelectPathButton()
if config_data["train_path"] is not None:
    train_path = widgets.Text(layout=widgets.Layout(width='30%'), value=config_data["train_path"])
    train_dir = config_data["train_path"]
else:
    train_path = widgets.Text(layout=widgets.Layout(width='30%'))

widgets.HBox([train_path, train_button])

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess 

style = {'description_width': '100px'}
layout = {'width': '400px'}

summaryStep = widgets.Text(description = 'Summary steps',value = "100", disabled=False, style=style, layout=layout)
checkpointStep = widgets.Text(description = 'Checkpoint Step',value = "500", disabled=False, style=style, layout=layout)
maxSteps = widgets.Text(description = 'Maximum Steps',value = "250000", disabled=False, style=style, layout=layout)
gpu_id = widgets.Text(description = 'GPU Id',value = "", disabled=False, style=style, layout=layout)

run_training_button = widgets.Button(description="Run Training", layout=widgets.Layout(width='20%', height='50px'))
training_out = widgets.Output()

def on_button_clicked(_):
    with training_out:
        clear_output()
        if train_dir is not None and data_dir is not None:
            run_training_button.disabled=True
            summary_step = int(summaryStep.value)
            checkpoint_step = int(checkpointStep.value)
            max_steps = int(maxSteps.value)
            if gpu_id.value != "":
                gpu = int(gpu_id.value)
            else:
                gpu = -1
            print('\033[1m' + "To stop the running training, click Kernel->Interrupt option from menubar")
            print('\033[0m')
            if gpu >= 0:
                !python src/train.py --dataset 'KITTI' --net 'squeezeDet' --data_path $data_dir --train_dir $train_dir --image_set 'train' --summary_step $summary_step --max_steps $max_steps --checkpoint_step $checkpoint_step --gpu $gpu
            else:
                !python src/train.py --dataset 'KITTI' --net 'squeezeDet' --data_path $data_dir --train_dir $train_dir --image_set 'train' --summary_step $summary_step --max_steps $max_steps --checkpoint_step $checkpoint_step
            run_training_button.disabled=False 
            with open(config_file, 'w') as outf:
                config_data["summary_step"] = str(summary_step)
                config_data["checkpoint_step"] = str(checkpoint_step)
                config_data["max_steps"] = str(max_steps)
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()
        else:
            print("Please select dataset and training directory")

run_training_button.on_click(on_button_clicked)

widgets.VBox([widgets.HBox([widgets.VBox([summaryStep, checkpointStep, maxSteps, gpu_id]), run_training_button]), training_out])

### Training Status

#### Tensorboard
TensorBoard provides us with a suite of web applications that help us to inspect and understand the TensorFlow runs and graphs. Currently, it provides five types of visualizations: scalars, images, audio, histograms, and graphs.

In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess, os

button_descriptions  = {False: "Stop Tensorboard", True: "Launch Tensorboard"}
p = None

def button_action(value):
    global p
    if train_dir is not None:
        if button.description == "Launch Tensorboard":
            owd = os.getcwd()
            os.chdir(train_dir)
            if [file for file in glob.glob("*.tfevents*")] != []:
                os.chdir(owd)
                cmd = ["tensorboard", "--logdir=" + str(train_dir)]
                p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                for line in p.stderr:
                    if line.decode("utf8").find("(Press CTRL+C to quit)") > 0:
                        print(line.decode("utf8").split("(Press CTRL+C")[0])
                        state = False
                        break
                    elif line.decode("utf8").find("could not bind to port 6006, it was already in use") > 0:
                        print("Port 6006 already in use, kill the process running on it and try again.")
                        state = True
                        break
                    else:
                        print(line.decode("utf8"))
                        state = True
            else:
                print("No event file found in train directory")
                state = True
        else:
            if p is not None:
                print("Tensorboard stopped")
                p.kill()
                state = True
        
        value.owner.description = button_descriptions[state]
    else:
        print("Please select train directory")


state = True

button = widgets.ToggleButton(state, description = button_descriptions[state], layout=widgets.Layout(width='20%', height='50px'))
button.observe(button_action, "value")

display(button)
print('\033[1m' + "Tensorboard will launch at 6006 port. If any process is running on that port please kill it.")
print('\033[0m')  

ToggleButton(value=True, description='Launch Tensorboard', layout=Layout(height='50px', width='20%'))

Tensorboard will launch at 6006 port. If any process is running on that port please kill it.

TensorBoard 1.14.0 at http://tanvi:6006/ 
Tensorboard stopped


### Model Freezing
#### Click the below button to generate the frozen pb from checkpoint and graph.pbtxt files. The pb file will easily pass through the sensAI tool. The generated pb file will be copied into the shared train directory so that it can be used in host environment.

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import tensorflow as tf
import os, glob

freeze_button = widgets.Button(description="Model Freezing", layout=widgets.Layout(width='20%', height='50px'))
freeze_out = widgets.Output()

def on_button_clicked(_):
    with freeze_out:
        clear_output()
        if train_dir is not None:
            print("Model freezing with the latest checkpoint")
            !python src/genpb.py --ckpt_dir $train_dir --freeze
            owd = os.getcwd()
            os.chdir(train_dir)
            !cp *.pb $SAVE_FILE_PATH
            os.chdir(owd)
            print("Model .pb generated and copied to train directory")
        else:
            print("Please select train directory")
             

# linking button and function together using a button's method
freeze_button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([freeze_button, freeze_out])